## Flurry number 2 - Carlando!

2020 was an amazing year to watch Forumla 1 for the first time. McLaren quickly stood out to me as my favorite team, with their laid back drivers and brilliant social media presence. But while their drivers Carlos Sainz and Lando Norris are good friends on and off track, it's time to see who stood out more this year, despite both of them having fantastic performances this year. We'll take a look at data from qualifying, races, and hopefully (if the F1 API allows it) lap times.

In [1]:
import numpy as np
import pandas as pd
import requests